In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
import numpy as np
import os

# Carga del dataset y preprocesamiento

In [ ]:
# Carga el dataset de AG News
ds, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.CBP384_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.CBP384_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Convierte el dataset a un formato utilizable
train_ds, test_ds = ds['train'], ds['test']

# Función para convertir un dataset de TensorFlow en una lista de Pandas
def tfds_to_pandas(tfds_dataset):
    texts = []
    labels = []
    for text, label in tfds_dataset:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    return pd.DataFrame({'text': texts, 'label': labels})

# Convierte el dataset de entrenamiento y prueba a DataFrames
train_df = tfds_to_pandas(train_ds)
test_df = tfds_to_pandas(test_ds)

In [ ]:
# Concatenar todos los textos en una sola cadena
all_text = ' '.join(train_df['text'].values)

# Dividir la cadena en palabras individuales
words = all_text.split()

# Obtener el vocabulario único de palabras
vocab = sorted(set(words))
vocab_size = len(vocab)
print(f'{vocab_size} unique words')

156039 unique words


Notamos que el dataset contiene muchas palabras únicas, decidimos reducirlo al 10% de las palabras que más se utilizan para que el entrenamiento no sea tan costoso.

In [ ]:
from collections import Counter

word_counts = Counter(words)

# Obtener el 10% de las palabras más frecuentes
top_10_percent_words = [word for word, count in word_counts.most_common(int(len(word_counts) * 0.10))]

# Crear un vocabulario único con estas palabras
vocab = sorted(set(top_10_percent_words))
vocab_size = len(vocab)
print(f'{vocab_size} unique words')

15603 unique words


In [ ]:
#convierte las palabras en IDs
ids_from_words = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)
#convierte los IDs en palabras
words_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

# Convertir palabras a IDs y viceversa
ids = ids_from_words(words)

# Generación del modelo

In [ ]:
class WordGenerationModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            x, state_h, state_c = self.lstm(x, training=training)
            states = [state_h, state_c]
        else:
            x, state_h, state_c = self.lstm(x, initial_state=states, training=training)
            states = [state_h, state_c]

        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

El modelo de generación de texto por palabras utiliza una red neuronal recurrente (RNN) compuesta por una capa de embedding para convertir palabras en vectores densos, una capa LSTM (Long Short-Term Memory) para capturar dependencias temporales en las secuencias de texto, y una capa densa final que proyecta la salida de la LSTM al espacio de palabras del vocabulario. El modelo está diseñado para recibir secuencias de palabras como entrada y predecir la siguiente palabra en la secuencia.

In [ ]:
# Función para generar secuencias de entrada y salida
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Convertir palabras a IDs y generar secuencias
all_ids = ids_from_words(words)
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Crear secuencias de entrenamiento
seq_length = 20
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True).map(split_input_target)

# Batch size y buffer size para el dataset
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Preparar el dataset para el entrenamiento
dataset = (
    sequences
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


In [ ]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 20), dtype=tf.int64, name=None), TensorSpec(shape=(64, 20), dtype=tf.int64, name=None))>

# Entrenamiento del modelo

In [ ]:

# Compilar el modelo y definir función de pérdida
vocab_size = len(ids_from_words.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

model = WordGenerationModel(vocab_size, embedding_dim, rnn_units)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Entrenar el modelo
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
2773/2773 [==============================] - 171s 60ms/step - loss: 5.8349
Epoch 2/20
2773/2773 [==============================] - 161s 58ms/step - loss: 4.8170
Epoch 3/20
2773/2773 [==============================] - 161s 58ms/step - loss: 4.4342
Epoch 4/20
2773/2773 [==============================] - 161s 58ms/step - loss: 4.1651
Epoch 5/20
2773/2773 [==============================] - 159s 57ms/step - loss: 3.9394
Epoch 6/20
2773/2773 [==============================] - 160s 58ms/step - loss: 3.7392
Epoch 7/20
2773/2773 [==============================] - 159s 57ms/step - loss: 3.5578
Epoch 8/20
2773/2773 [==============================] - 160s 57ms/step - loss: 3.3915
Epoch 9/20
2773/2773 [==============================] - 159s 57ms/step - loss: 3.2388
Epoch 10/20
2773/2773 [==============================] - 160s 57ms/step - loss: 3.0978
Epoch 11/20
2773/2773 [==============================] - 159s 57ms/step - loss: 2.9679
Epoch 12/20
2773/2773 [=============================

Observamos que el modelo fue mejorando la métrica de la función de perdida en cada época. Con más épocas entendemos que se seguiría optimizando. La elección de entrenarlo con 20 épocas fue exclusivamente una cuestión de tiempos.

# Prueba del modelo

In [ ]:
class OneStepWordGeneration(tf.keras.Model):
    def __init__(self, model, words_from_ids, ids_from_words, temperature=0.2):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.words_from_ids = words_from_ids
        self.ids_from_words = ids_from_words

        # Crear una máscara para evitar que se genere "[UNK]".
        skip_ids = self.ids_from_words(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Poner un -inf en cada índice incorrecto.
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            # Igualar la forma al vocabulario
            dense_shape=[len(ids_from_words.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convertir la cadena de entrada a IDs de tokens.
        input_words = tf.strings.split(inputs)
        input_ids = self.ids_from_words(input_words).to_tensor()

        # Ejecutar el modelo para predecir la siguiente palabra.
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :]  # Usar solo la última predicción en la secuencia.
        predicted_logits /= self.temperature
        predicted_logits += self.prediction_mask  # Aplicar la máscara de predicción para evitar la generación de "[UNK]".

        # Muestrear los logits de salida para generar IDs de tokens.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convertir de IDs de tokens a palabras.
        predicted_words = self.words_from_ids(predicted_ids)

        # Devolver las palabras predichas y los estados del modelo.
        return predicted_words, states

In [ ]:
one_step_model = OneStepWordGeneration(model, words_from_ids, ids_from_words)

In [ ]:
# Lista de semillas para generar noticias
start_words_list = [
    "Breaking news:",
    "In sports",
    "The latest economic figures show",
    "Today's weather forecast predicts",
    "In a recent political development,"
]

# Lista para almacenar las noticias generadas
generated_news = []

# Generar noticias para cada semilla en la lista
for start_words in start_words_list:
    states = None
    next_words = tf.constant([start_words])
    result = [next_words]

    for _ in range(40):
        next_words, states = one_step_model.generate_one_step(next_words, states=states)
        result.append(next_words)

    result = tf.strings.reduce_join(result, separator=' ')
    generated_news.append(result.numpy().decode('utf-8'))

# Imprimir todas las noticias generadas
for news in generated_news:
    print(news)
    print("\n" + "_"*80 + "\n")  # Separador entre noticias

Breaking news: and the company #39;s leading scientific research institutions are on the verge of being a new beta for the MSN Messenger 7.0 which includes tools for free access to the existing application and application software application that will compete with

________________________________________________________________________________

In sports features that he wants to play in the majors with a dramatic victory in the Olympic women #39;s hockey tournament on Sunday. The US gymnast who spun into the gold medal in the women #39;s marathon on Sunday was a

________________________________________________________________________________

The latest economic figures show of the dangers of the international consortium is to be the most capable member of the country #39;s largest oil producer by the end of next year and save thousands of jobs and build a new stadium in the United

________________________________________________________________________________

Today's weather fo

El modelo entrenado muestra una capacidad para construir frases coherentes y mantener el contexto en varios temas. Aunque algunas oraciones pueden parecer fragmentadas o repetitivas, el modelo logra producir contenido relevante y con sentido, mencionando eventos específicos y entidades conocidas. Las respuestas incluyen detalles específicos y tecnicismos que reflejan un conocimiento básico de la estructura de noticias, aunque con ciertas limitaciones en la coherencia y fluidez global del texto generado. Comparando los modelos, entendemos que el de generación por palabras se muestra un poco más robusto en comparación al de caracteres, inferimos que es porque al ser modelos sencillos, el de caracteres puede inventar palabras que no existen y el de palabras puede adaptarse mejor al contexto de la secuencia.